In [1]:
import pandas as pd
from collections import Counter
import time
# from data_test import *
# from mushroom import *
# from retail_transaction_dataset import *
from fruithut.fruithut import *

     Tid                                       Items
0     T1                                [1079, 2032]
1     T2                                      [2032]
2     T3                                [1001, 4019]
3     T4                                      [1001]
4     T5                          [1001, 1003, 1123]
..   ...                                         ...
95   T96                          [1032, 1071, 2030]
96   T97  [1002, 1008, 1017, 1063, 2003, 2010, 2030]
97   T98                                      [2036]
98   T99                          [1058, 2010, 2030]
99  T100                                [1033, 2022]

[100 rows x 2 columns]


In [2]:

# sample
# data = {
#     'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
#     'Items': [['a', 'c', 'd'],
#               ['a', 'b', 'd'],
#               ['b', 'c', 'd', 'e'],
#               ['a', 'd'],
#               ['c', 'd', 'e'],
#               ['a', 'b', 'c', 'd', 'e']]
# }
# df = pd.DataFrame(data)

# data = {
#     'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
#     'Items': [['apple', 'cherry', 'durian'],
#               ['apple', 'banana', 'durian'],
#               ['banana', 'cherry', 'durian', 'elderberry'],
#               ['apple', 'durian'],
#               ['cherry', 'durian', 'elderberry'],
#               ['apple', 'banana', 'cherry', 'durian', 'elderberry']]
# }

# df = pd.DataFrame(data)
df['Item_Length'] = df['Items'].apply(lambda items: len(items))
len_df = len(df)
df

,Tid,Items,Item_Length
0,T1,"[1079, 2032]",2
1,T2,[2032],1
2,T3,"[1001, 4019]",2
3,T4,[1001],1
4,T5,"[1001, 1003, 1123]",3
...,...,...,...
95,T96,"[1032, 1071, 2030]",3
96,T97,"[1002, 1008, 1017, 1063, 2003, 2010, 2030]",7
97,T98,[2036],1
98,T99,"[1058, 2010, 2030]",3


In [3]:
unique_items = sorted(df['Items'].explode().unique()) # get unique item => save to list
unique_items

['1001',
 '1002',
 '1003',
 '1004',
 '1007',
 '1008',
 '1012',
 '1014',
 '1017',
 '1021',
 '1027',
 '1031',
 '1032',
 '1033',
 '1034',
 '1037',
 '1054',
 '1058',
 '1063',
 '1064',
 '1066',
 '1071',
 '1073',
 '1076',
 '1077',
 '1078',
 '1079',
 '1080',
 '1099',
 '1109',
 '1111',
 '1123',
 '1203',
 '2002',
 '2003',
 '2004',
 '2005',
 '2010',
 '2021',
 '2022',
 '2023',
 '2025',
 '2028',
 '2030',
 '2031',
 '2032',
 '2034',
 '2036',
 '2037',
 '2038',
 '2041',
 '2045',
 '3013',
 '4001',
 '4019',
 '4024',
 '4031']

In [4]:
# calculate occupancy list: {'a': [(T1, 3), (T2, 3), (T4, 2), (T6, 5)]} - list Tid, len(Tid) containing unique item
def occupancy_list(df):
    occupancy_list = {} 
    for item in unique_items:
        tid_list = df[df['Items'].apply(lambda items: item in items)]['Tid'].tolist() # create column Items with items in unique item
        tid_lengths = [(tid, len(df[df['Tid'] == tid]['Items'].iloc[0])) for tid in tid_list if item in df[df['Tid'] == tid]['Items'].iloc[0]] # length of Tid containing unique items
        occupancy_list[item] = tid_lengths # add value with item_key
    df_occupancy_list = pd.DataFrame(occupancy_list.items(), columns=['Items', 'Occupancy_list'])
    df_occupancy_list['Items'] = df_occupancy_list['Items'].apply(lambda x: [x]) 
    return df_occupancy_list

In [5]:
df_occupancy_list = occupancy_list(df)
df_occupancy_list

,Items,Occupancy_list
0,[1001],"[(T3, 2), (T4, 1), (T5, 3), (T7, 4), (T8, 3)]"
1,[1002],"[(T14, 6), (T15, 3), (T22, 9), (T24, 14), (T32..."
2,[1003],"[(T5, 3), (T9, 4)]"
3,[1004],"[(T7, 4), (T9, 4)]"
4,[1007],"[(T8, 3), (T9, 4)]"
5,[1008],"[(T22, 9), (T43, 3), (T66, 6), (T81, 1), (T94,..."
6,[1012],"[(T82, 4)]"
7,[1014],"[(T8, 3)]"
8,[1017],"[(T62, 1), (T64, 4), (T97, 7)]"
9,[1021],"[(T11, 4), (T24, 14), (T40, 1), (T64, 4)]"


In [6]:
# calculate stset: {'a': [T1, T2, T4, T6]} - list of Tid containing unique item
def cal_stset(df_occupancy_list):
    df_stset = pd.DataFrame(columns=['Items', 'Occupancy'])

    for index, row in df_occupancy_list.iterrows():
        item = row['Items']
        occupancy_list = [tid for tid, _ in row['Occupancy_list']] # get Tid containing unique item
        df_stset = df_stset.append({'Items': item, 'Occupancy': occupancy_list}, ignore_index=True)
    return df_stset

In [7]:
df_stset = cal_stset(df_occupancy_list)
df_stset

,Items,Occupancy
0,[1001],"[T3, T4, T5, T7, T8]"
1,[1002],"[T14, T15, T22, T24, T32, T35, T41, T57, T85, ..."
2,[1003],"[T5, T9]"
3,[1004],"[T7, T9]"
4,[1007],"[T8, T9]"
5,[1008],"[T22, T43, T66, T81, T94, T97]"
6,[1012],[T82]
7,[1014],[T8]
8,[1017],"[T62, T64, T97]"
9,[1021],"[T11, T24, T40, T64]"


In [8]:
# calculate support - count number of Tid containing unique item
def cal_support(df_stset):
    df_support = pd.DataFrame(columns=['Items', 'Support'])
    df_support['Items'] = df_stset['Items']
    df_support['Support'] = df_stset['Occupancy'].apply(len)
    return df_support

In [9]:
df_support = cal_support(df_stset)
df_support

,Items,Support
0,[1001],5
1,[1002],11
2,[1003],2
3,[1004],2
4,[1007],2
5,[1008],6
6,[1012],1
7,[1014],1
8,[1017],3
9,[1021],4


In [10]:
# calculate occupancy - O(P) = ∑ T ∈ STSet(P) |P|/|T|
# |P|: len(unique item) itemset {a} =>1
# |T|: len(Tid) 1/3 + 1/3 + 1/2 + 1/5 
def cal_occupancy(df_occupancy_list):
    occupancy_data = []
    for index, row in df_occupancy_list.iterrows():
        item = row['Items']
        occupancy_list = row['Occupancy_list']
        total = 0
        for tid, length in occupancy_list:
            total += len(item) / length
        occupancy_data.append({'Items': item, 'Occupancy': round(total, 2)})
    
    df_occupancy = pd.DataFrame(occupancy_data)
    
    return df_occupancy

In [11]:
df_occupancy = cal_occupancy(df_occupancy_list)
df_occupancy


,Items,Occupancy
0,[1001],2.42
1,[1002],3.18
2,[1003],0.58
3,[1004],0.50
4,[1007],0.58
5,[1008],2.00
6,[1012],0.25
7,[1014],0.33
8,[1017],1.39
9,[1021],1.57


In [12]:
# ex: 'a': {'l(a)': [2, 3, 5], 'n(a)': [1, 2, 1]}
def df_prepare_UBO(df_occupancy_list):
    UBO_data = []
    for index, row in df_occupancy_list.iterrows():
        item = row['Items']
        occupancy_list = row['Occupancy_list']
        
        values = [i[1] for i in occupancy_list]
        l_item = sorted(set(values)) # get unique len(Tid) => sort ascending
    
        counter = Counter(values)
        n_item = [counter[i] for i in l_item] # count unique len(Tid) in occupancy_list => same index with l_item
        
        UBO_data.append({'Items': item, 'l_item': l_item, 'n_item': n_item})
    
    df_UBO = pd.DataFrame(UBO_data)
    return df_UBO

In [13]:
df_prepare_UBO(df_occupancy_list)

,Items,l_item,n_item
0,[1001],"[1, 2, 3, 4]","[1, 1, 2, 1]"
1,[1002],"[1, 2, 3, 4, 5, 6, 7, 9, 14]","[1, 1, 1, 1, 3, 1, 1, 1, 1]"
2,[1003],"[3, 4]","[1, 1]"
3,[1004],[4],[2]
4,[1007],"[3, 4]","[1, 1]"
5,[1008],"[1, 3, 4, 6, 7, 9]","[1, 1, 1, 1, 1, 1]"
6,[1012],[4],[1]
7,[1014],[3],[1]
8,[1017],"[1, 4, 7]","[1, 1, 1]"
9,[1021],"[1, 4, 14]","[1, 2, 1]"


In [14]:
# calculate according to the formula: ni x lx/li
def cal_ubo(l, n):
    total = 0
    for i in range(len(l)):
        total += n[i] * l[0] / l[i]
    return round(total, 2)

In [15]:
# summarize: ∑ni x lx/li => save to list 
def ubo_final(length, number_transaction):
    ubo = []
    for i in range(len(length)): 
        # ex: len = [2,3,5], num_trans = [1,2,1]
        # i = 0 => len = [2,3,5], num_trans = [1,2,1]
        # i = 1 => len = [3,5], num_trans = [2,1]
        # ...
        ubo.append(cal_ubo(length[i:], number_transaction[i:])) # save result cal_ubo for each i => get maxUBO
    return ubo

In [16]:
# get max from summarize => save max value in UBO by key
def calculate_maxUBO(df_UBO):
    df_UBO['List_UBO'] = None # create new column
    df_UBO['Max_UBO'] = None # create new column
    for index, row in df_UBO.iterrows():
        length = row['l_item'] #get list of len(Tid) containing unique item
        number_transaction = row['n_item'] # count unique len(Tid) in occupancy_list
        
        ubo = ubo_final(length, number_transaction) # get list of UBO by i. ex: [2.73, 2.6, 1.0]
        max_ubo = max(ubo) # max list of UBO
        
        df_UBO.at[index, 'List_UBO'] = ubo # save result in df
        df_UBO.at[index, 'Max_UBO'] = max_ubo # save result in df
        
    return df_UBO

In [17]:
# UBO calculation methods: main function
def cal_UBO(df_occupancy_list): 
    df_UBO = df_prepare_UBO(df_occupancy_list)    
    df_UBO = calculate_maxUBO(df_UBO)
    return df_UBO

In [18]:
df_UBO = cal_UBO(df_occupancy_list)
df_UBO

,Items,l_item,n_item,List_UBO,Max_UBO
0,[1001],"[1, 2, 3, 4]","[1, 1, 2, 1]","[2.42, 2.83, 2.75, 1.0]",2.83
1,[1002],"[1, 2, 3, 4, 5, 6, 7, 9, 14]","[1, 1, 1, 1, 3, 1, 1, 1, 1]","[3.18, 4.35, 5.03, 5.37, 5.46, 2.95, 2.28, 1.6...",5.46
2,[1003],"[3, 4]","[1, 1]","[1.75, 1.0]",1.75
3,[1004],[4],[2],[2.0],2
4,[1007],"[3, 4]","[1, 1]","[1.75, 1.0]",1.75
5,[1008],"[1, 3, 4, 6, 7, 9]","[1, 1, 1, 1, 1, 1]","[2.0, 3.01, 2.68, 2.52, 1.78, 1.0]",3.01
6,[1012],[4],[1],[1.0],1
7,[1014],[3],[1],[1.0],1
8,[1017],"[1, 4, 7]","[1, 1, 1]","[1.39, 1.57, 1.0]",1.57
9,[1021],"[1, 4, 14]","[1, 2, 1]","[1.57, 2.29, 1.0]",2.29


In [19]:
# runtime
def runtime():
    start_time = time.time()
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

In [20]:
# the index, support, occupancy and UBO of each itemset
def itemset_info(df_occupancy, df_support, df_UBO):
    df_occupancy_exploded = df_occupancy.explode('Items')
    df_support_exploded = df_support.explode('Items')
    df_UBO_exploded = df_UBO.explode('Items')
    merge_df = df_support_exploded[['Items', 'Support']].merge(df_occupancy_exploded[['Items', 'Occupancy']], on = "Items").\
        merge(df_UBO_exploded[['Items', 'Max_UBO']], on = "Items")
    merge_df['Items'] = merge_df['Items'].apply(lambda x: [x]) 
    
    return merge_df

In [21]:
df_itemset_info = itemset_info(df_occupancy, df_support, df_UBO)
df_itemset_info

,Items,Support,Occupancy,Max_UBO
0,[1001],5,2.42,2.83
1,[1002],11,3.18,5.46
2,[1003],2,0.58,1.75
3,[1004],2,0.50,2
4,[1007],2,0.58,1.75
5,[1008],6,2.00,3.01
6,[1012],1,0.25,1
7,[1014],1,0.33,1
8,[1017],3,1.39,1.57
9,[1021],4,1.57,2.29


In [22]:
def df_intersection(items1, items2, df_occupancy_list):    
    df = pd.DataFrame(columns=['Items', 'Occupancy_list'])
    set1 = set(items1)
    set2 = set(items2)
    list_1_item = sorted(list(set1 | set2))
    
    list_occupancy_1_item = []
    
    for i in list_1_item:
        list_occupancy_1_item.append(df_occupancy_list[df_occupancy_list['Items'].apply(lambda item: i in item)]["Occupancy_list"].iloc[0])
        
    intersection_list = set(list_occupancy_1_item[0])
    for sublist in list_occupancy_1_item[1:]:
        intersection_list = intersection_list.intersection(sublist)

    intersection_list = sorted(intersection_list, key = lambda x: x[0])
    
    df = df.append({'Items': list_1_item, 'Occupancy_list': intersection_list}, ignore_index=True)

    return df

In [23]:
def hep_algorithm(threshold, len_df, df_itemset_info, df_occupancy_list):
    threshold = threshold * len_df # ex: threshold = 25% of len(database)
    C1 = []
    HO1 = []
    
    for index, row in df_itemset_info.iterrows():
        items = row['Items'] # 1-itemset in row
        support = row['Support'] # support of 1-itemset
        occupancy = row['Occupancy'] # occuopancy of 1-itemset
        max_ubo = row['Max_UBO'] # max_ubo of 1-itemset
        
        if support >= threshold:
            if max_ubo >= threshold:
                # C1.add(frozenset(items)) # candidate 1-itemset => use item for C1 to create k-itemset (k = 2, k += 1 for each loop)
                C1.append(items)
            if occupancy >= threshold:
                # HO1.add(frozenset(items)) # High Occupancy 1-itemset.
                HO1.append(items)

    # return C1, HO1
    
    # Generate Ck and HOk
    k = 2
    Ck_1 = C1
    HOk = HO1
    
    while Ck_1:
        Ck = []
        for i in range(len(Ck_1)):
            for j in range(i+1, len(Ck_1)):
                if len(set(Ck_1[i]) & set(Ck_1[j])) == k - 2: # # C1 = {{'a'},{'b'},{'c'},{'d'},{'e'}}
                    # P = set(set(Ck_1[i]).union(set(Ck_1[j])))
                    # print(P)
                    P_occupancy_list = df_intersection(Ck_1[i], Ck_1[j], df_occupancy_list)
                    p_items = P_occupancy_list['Items'].iloc[0]
                    if p_items in HOk:
                        continue
                    # if any(p_items == item for item in HOk):
                    #     continue
                    p_ubo = 0
                    try:
                        p_ubo = cal_UBO(P_occupancy_list)['Max_UBO'].iloc[0]
                    except ValueError:
                        # p_ubo = 0
                        continue
                    p_occupancy = cal_occupancy(P_occupancy_list)['Occupancy'].iloc[0]
                    if p_ubo >= threshold:
                        Ck.append(p_items)
                        if p_occupancy >= threshold:
                            HOk.append(p_items)
        k += 1
        Ck_1 = Ck
        
    # Return the set of all high occupancy itemsets
    # HO_Set = HOk
    # for itemsets in HOk:
    #     HO_Set.append(itemsets)
    
    return HOk            

In [26]:
threshold = 0.01
HO_Set = hep_algorithm(threshold, len_df, df_itemset_info, df_occupancy_list)
for i in HO_Set:
    print(i)
print(len(HO_Set))
runtime()

KeyboardInterrupt: 